<a href="https://colab.research.google.com/github/TerjeNorderhaug/patient1/blob/master/Undiagnosed_1_Workbench.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Undiagnosed-1 Workbench by Patient1



## Install Tensorflow 2

Using Tensorflow Nightly Preview as  Clairvoyante  is incompatible with 2.0.0-alpha0

In [0]:
!pip install tf-nightly-gpu-2.0-preview

import tensorflow as tf
print(tf.__version__)

## Install Nucleus

[Nucleus](https://github.com/google/nucleus) is a library of Python code for reading, writing, and filtering common genomics file formats for conversion to TensorFlow examples. This installation works with TF2.



In [0]:
!pip install -q google-nucleus==0.2.2  # Other versions refuses TF2

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import random

import numpy as np
import tensorflow as tf

from nucleus.io import fasta
from nucleus.io import sam
from nucleus.io import vcf
from nucleus.io.genomics_writer import TFRecordWriter
from nucleus.protos import reads_pb2
from nucleus.util import cigar
from nucleus.util import ranges
from nucleus.util import utils

## Clairvoyante TF2 Update

[Clairvoyante](https://github.com/aquaskyline/Clairvoyante) is a Tensorflow based variant caller, a multi-task five-layer convolutional neural network model for predicting variant type, zygosity, alternative allele and Indel length. **We have upgraded it for Tensorflow 2.**

In [0]:
import os
! rm -r Clairvoyante_download
! git clone --depth=1 https://github.com/aquaskyline/Clairvoyante.git Clairvoyante_download
print(os.listdir('./Clairvoyante_download'))

In [0]:
!rm -r ./Clairvoyante
!rm -r ./Clairvoyante_tmp
!cp -r ./Clairvoyante_download ./Clairvoyante_tmp
# tf_upgrade_v2 script assumes tensorflow is imported as 'tf' for pattern matching
!find ./Clairvoyante_tmp -type f -name "*.py" -print0 | xargs -0 sed -i '' -e 's/tensorflow\./tf\./g'
!tf_upgrade_v2 --intree ./Clairvoyante_tmp --outtree ./Clairvoyante

In [0]:
import os
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow.python.framework
import tensorflow.python.ops
from tensorflow.python.layers import utils
from tensorflow.keras.initializers import VarianceScaling

!find ./Clairvoyante -type f -name "selu.py" -print0 | xargs -0 sed -i '' -e 's/from tf.contrib import layers/from tensorflow.keras.initializers import VarianceScaling/g'
!find ./Clairvoyante -type f -name "selu.py" -print0 | xargs -0 sed -i '' -e "s/layers.variance_scaling_initializer(factor=1.0, mode='FAN_IN')/VarianceScaling\(scale=1.0, mode='fan_in'\)/g"
!find ./Clairvoyante -type f -name "clairvoyante_v3.py" -print0 | xargs -0 sed -i '' -e "s/import selu/import selu\nfrom tensorflow.keras.initializers import VarianceScaling/g"
!find ./Clairvoyante -type f -name "clairvoyante_v3.py" -print0 | xargs -0 sed -i '' -e "s/tf.contrib.layers.variance_scaling_initializer(factor=2.0, mode='FAN_IN', uniform=False)/VarianceScaling(scale=1.0, mode='fan_in', distribution='truncated_normal')/g"

# Missed by the upgrade script:
!find ./Clairvoyante -type f -name "*.py" -print0 | xargs -0 sed -i '' -e 's/tf.python.framework/tensorflow.python.framework/g'
!find ./Clairvoyante -type f -name "*.py" -print0 | xargs -0 sed -i '' -e 's/tf.python.ops/tensorflow.python.ops/g'
!find ./Clairvoyante -type f -name "*.py" -print0 | xargs -0 sed -i '' -e 's/tf.contrib.layers.python.layers/tensorflow.python.layers/g'
!find ./Clairvoyante -type f -name "*.py" -print0 | xargs -0 sed -i '' -e 's/tf.contrib.layers.python.layers/tensorflow.python.layers/g'


print(os.listdir('./Clairvoyante'))
!cat "./Clairvoyante/clairvoyante/selu.py"

In [0]:
# !pip install --upgrade setuptools pip
!pip install blosc
!pip install intervaltree==2.1.0
!pip install numpy

In [0]:
import os
import sys
print(os.listdir('.'))
print(os.listdir('./Clairvoyante'))
print(os.listdir('./Clairvoyante/clairvoyante'))
sys.path.append('./Clairvoyante')
# sys.path.append('./Clairvoyante/clairvoyante')

### Accelerate with PyPy

In [0]:
!apt-get -y install pypy samtools wget pypy-dev parallel
!wget https://bootstrap.pypa.io/get-pip.py
!sudo pypy get-pip.py 
!sudo pypy -m pip install intervaltree 

# Variant Calling

The data for Undiagnosed-1 is only available for the participants and could not be included in this public notebook. Substitute with the data links below.

In [0]:
!wget 'http://www.bio8.cs.hku.hk/training.tar'
!tar -xf training.tar

!cd Clairvoyante
!curl http://www.bio8.cs.hku.hk/trainedModels.tbz | tar -jxf -
  
print(os.listdir('./trainedModels/fullv3-illumina-novoalign-hg001+hg002-hg38/'))
trained_model = "../trainedModels/fullv3-illumina-novoalign-hg001+hg002-hg38/learningRate1e-3.epoch500"
tensor_file = "tensor_can_chr21"
variant_calls_file = "tensor_can_chr21.vcf"


In [0]:
import os
import sys
import clairvoyante.utils_v2
import clairvoyante.clairvoyante_v3
import clairvoyante.callVar
from mock import patch


def call_variants (trained_model, tensor_file, variant_calls_file):
  args = \
  ["callVar.py",
   "--chkpnt_fn", trained_model, \
   "--tensor_fn", tensor_file, \
   "--call_fn", variant_calls_file]

  try:
    os.chdir("./training")
    with patch.object(sys, 'argv', args):
      clairvoyante.callVar.main()
  finally:
    os.chdir("..")
 

call_variants (trained_model, tensor_file, variant_calls_file)

### Verify variants



In [0]:
with open('./training/tensor_can_chr21.vcf', mode='r') as vcf:
    print(vcf.read())

## View Variants

In [0]:
import matplotlib.pyplot as plt
import numpy as np
import clairvoyante.utils_v2 as utils
from clairvoyante.getTensorAndLayerPNG import PlotTensor

# Load the trained dataset
total, XArrayCompressed, YArrayCompressed, posArrayCompressed = \
  utils.GetTrainingArray("./training/tensor_can_chr21",
                         "./training/var_chr21",
                         "./training/bed",
                         shuffle = False)

def show_variant_tensors (i = 0):
  XArray, _, _ = utils.DecompressArray(XArrayCompressed, i, 1, total)
  fig = plt.figure(figsize=(15, 8));
  fig.suptitle("Variant {0}".format(i), fontsize=16)
  def tensor_subplot(ix, vmin=0, cmap=plt.cm.hot):
    plt.subplot(4,1,ix); 
    plt.xticks(np.arange(0, 33, 1)); 
    plt.yticks(np.arange(0, 4, 1), ['A','C','G','T'])
    plt.imshow(XArray[0,:,:,ix-1].transpose(), vmin=vmin, vmax=50, interpolation="nearest", cmap=cmap); 
    plt.colorbar()
  tensor_subplot(1);
  tensor_subplot(2, vmin=-50, cmap=plt.cm.bwr)
  tensor_subplot(3, vmin=-50, cmap=plt.cm.bwr)
  tensor_subplot(4, vmin=-50, cmap=plt.cm.bwr)
  plt.show()

print("Total:", total)

In [0]:
show_variant_tensors(1000)
show_variant_tensors(1002)

for i in range(10,20):
  show_variant_tensors(i)

## Inspect Variants With Nucleus

In [0]:
from nucleus.io import vcf

with vcf.VcfReader('./training/tensor_can_chr21.vcf') as reader:
  print('Sample names in VCF: ', ' '.join(reader.header.sample_names))
  with vcf.VcfWriter('/tmp/filtered.tfrecord', header=reader.header) as writer:
    for variant in reader:
      print(variant)
      if variant.quality > 3.01:
        writer.write(variant)

## Variant Calling On Generated Data

Grab a coffee while you wait...

In [0]:
!wget 'http://www.bio8.cs.hku.hk/testingData.tar'
!tar -xf testingData.tar

print(os.listdir('.'))

!cd dataPrepScripts
!sh PrepDataBeforeDemo.sh

In [0]:
import sys
import clairvoyante.callVarBam
from mock import patch

print(os.getcwd())

print(os.listdir('.'))
print("Trained Models:", os.listdir('./trainedModels/'))
print(os.listdir('./trainedModels/fullv3-illumina-novoalign-hg001+hg002-hg38/'))
print("TestingData:", os.listdir('./testingData/chr21'))
print("Training:", os.listdir('./training/'))

def call_var_bam(outfile):
  # Hack to override sysargs.
  args = \
  ["callVarBam.py", 
   "--chkpnt_fn", "../trainedModels/fullv3-illumina-novoalign-hg001+hg002-hg38/learningRate1e-3.epoch500", \
   "--bam_fn", "../testingData/chr21/chr21.bam", \
   "--ref_fn", "../testingData/chr21/chr21.fa", \
   "--bed_fn", "../testingData/chr21/chr21.bed", \
   "--call_fn", outfile,  \
   "--ctgName", "chr21"]
  try:
    os.chdir("./training")
    with patch.object(sys, 'argv', args):
      clairvoyante.callVarBam.main()
  finally:
    os.chdir("..")
  

call_var_bam("chr21_calls.vcf")